In [1]:
import pandas as pd
import sqlite3

# 1. Establish a connection to the SQLite database
# A file named 'sales_analysis.db' will be created if it doesn't exist.
conn = sqlite3.connect('sales_analysis.db')

print("Database connection established successfully.")

Database connection established successfully.


In [2]:
FILE_NAME = 'sales_store.csv' 

try:
    # 2. Load the data from the uploaded CSV file
    df = pd.read_csv(FILE_NAME) 
    
    # 3. Write the DataFrame to a SQL table named 'sales_data'
    df.to_sql('sales_data', conn, if_exists='replace', index=False)
    
    print(f"Data from '{FILE_NAME}' successfully written to SQL table 'sales_data'.")
    
    # Inspect the first few rows to confirm column names
    print("\nFirst 5 rows of the data (Use these column names for your SQL query):")
    print(df.head())
    
except FileNotFoundError:
    print(f"Error: '{FILE_NAME}' not found. Please ensure the file is uploaded to the same directory as this notebook.")

Data from 'sales_store.csv' successfully written to SQL table 'sales_data'.

First 5 rows of the data (Use these column names for your SQL query):
  transaction_id customer_id   customer_name  customer_age  gender product_id  \
0      TXN886726    CUST4481  Dhanush Bhakta          41.0       F      P8464   
1      TXN941069    CUST9221  Anahita Shenoy          46.0       F      P7932   
2      TXN132714    CUST9088    Samaira Bora          29.0  Female      P9223   
3      TXN855235    CUST5938   Yashvi Sachar          44.0    Male      P6308   
4      TXN570157    CUST1985      Rhea Manda          45.0  Female      P5718   

  product_name product_category  quantiy   prce payment_mode purchase_date  \
0         Sofa        Furniture        5  21410          UPI    29-08-2023   
1   Sunglasses      Accessories        2   1156         Cash    04-06-2023   
2   Sunglasses      Accessories        4  13828          UPI    16-09-2023   
3   Smartwatch      Electronics        3  11214       

In [7]:
print(df.columns)

Index(['transaction_id', 'customer_id', 'customer_name', 'customer_age',
       'gender', 'product_id', 'product_name', 'product_category', 'quantiy',
       'prce', 'payment_mode', 'purchase_date', 'time_of_purchase', 'status'],
      dtype='object')


In [ ]:

check_query = """
SELECT 
    purchase_date 
FROM 
    sales_data 
LIMIT 5;
"""
date_check_df = pd.read_sql_query(check_query, conn)
print("--- Sample Purchase Dates ---")
print(date_check_df)

--- Sample Purchase Dates ---
  purchase_date
0    29-08-2023
1    04-06-2023
2    16-09-2023
3    26-08-2023
4    23-11-2023


In [29]:

sql_query = """
SELECT
    -- 1. Create the YYYY-MM component directly for selection and grouping
    STRFTIME(
        '%Y-%m', 
        SUBSTR(purchase_date, 7, 4) || '-' || 
        SUBSTR(purchase_date, 4, 2) || '-' || 
        SUBSTR(purchase_date, 1, 2)
    ) AS Order_YearMonth,
    
    -- 2. Aggregate Total Revenue and Transaction Volume
    SUM(prce * quantiy) AS Total_Monthly_Revenue,
    COUNT(DISTINCT transaction_id) AS Total_Monthly_Transactions
FROM
    sales_data
GROUP BY
    Order_YearMonth  -- <-- The alias IS now usable here because the alias creation is defined directly above it, and it's the simplest way in Pandas/SQLite.
    
ORDER BY
    Order_YearMonth DESC;
"""

# Let's try the more explicit, robust way if the simple alias fails again:
sql_query_explicit = """
SELECT
    STRFTIME(
        '%Y-%m', 
        SUBSTR(purchase_date, 7, 4) || '-' || SUBSTR(purchase_date, 4, 2) || '-' || SUBSTR(purchase_date, 1, 2)
    ) AS Order_YearMonth,
    
    SUM(prce * quantiy) AS Total_Monthly_Revenue,
    COUNT(DISTINCT transaction_id) AS Total_Monthly_Transactions
FROM
    sales_data
GROUP BY
    STRFTIME(
        '%Y-%m', 
        SUBSTR(purchase_date, 7, 4) || '-' || SUBSTR(purchase_date, 4, 2) || '-' || SUBSTR(purchase_date, 1, 2)
    ) -- <-- REPEATING the full logic here ensures scope is not an issue
ORDER BY
    Order_YearMonth DESC;
"""

try:
    analysis_df = pd.read_sql_query(sql_query_explicit, conn)
    
    print("\n--- Final Successful Monthly Revenue Analysis ---")
    print(analysis_df)

except Exception as e:
    print(f"An error occurred with the explicit query: {e}")


--- Final Successful Monthly Revenue Analysis ---
   Order_YearMonth  Total_Monthly_Revenue  Total_Monthly_Transactions
0          2024-01                 339442                           9
1          2023-12                5249987                         166
2          2023-11                5125709                         174
3          2023-10                5886414                         178
4          2023-09                5057387                         170
5          2023-08                4825701                         166
6          2023-07                5129904                         189
7          2023-06                4100112                         162
8          2023-05                3902263                         141
9          2023-04                4989315                         161
10         2023-03                5241364                         144
11         2023-02                4698929                         176
12         2023-01                46286

In [21]:


# Define the SQL query to analyze sales by gender
sql_query_gender = """
SELECT
    gender AS Customer_Gender,
    
    -- Aggregate Revenue and Transaction Volume
    SUM(prce * quantiy) AS Total_Gender_Revenue,
    COUNT(DISTINCT transaction_id) AS Total_Gender_Transactions,
    
    -- Calculate AOV for the gender group
    CAST(SUM(prce * quantiy) AS REAL) / COUNT(DISTINCT transaction_id) AS Average_Order_Value
    
FROM
    sales_data
GROUP BY
    Customer_Gender
ORDER BY
    Total_Gender_Revenue DESC;
"""

# Assuming 'conn' is your active database connection from the initial steps
try:
    analysis_df_gender = pd.read_sql_query(sql_query_gender, conn)
    
    print("--- Sales Analysis by Gender ---")
    print(analysis_df_gender)

except Exception as e:
    print(f"An error occurred during query execution: {e}")
    print("\nEnsure your database connection ('conn') is still active and the column names are correct.")

--- Sales Analysis by Gender ---
  Customer_Gender  Total_Gender_Revenue  Total_Gender_Transactions  \
0            Male              14938253                        513   
1               F              14821961                        479   
2          Female              14814778                        504   
3               M              14593105                        503   
4            None                 66190                          1   

   Average_Order_Value  
0         29119.401559  
1         30943.551148  
2         29394.400794  
3         29012.137177  
4         66190.000000  


In [ ]:
sql_query_aov = """
SELECT
    -- 1. Date conversion for grouping
    STRFTIME(
        '%Y-%m', 
        SUBSTR(purchase_date, 7, 4) || '-' || SUBSTR(purchase_date, 4, 2) || '-' || SUBSTR(purchase_date, 1, 2)
    ) AS Order_YearMonth,
    
    -- 2. Aggregation: Revenue and Transaction Volume
    SUM(prce * quantiy) AS Total_Monthly_Revenue,
    COUNT(DISTINCT transaction_id) AS Total_Monthly_Transactions,
    
    -- 3. CALCULATE AOV: Total Revenue / Total Transactions
    CAST(SUM(prce * quantiy) AS REAL) / COUNT(DISTINCT transaction_id) AS Average_Order_Value
    
FROM
    sales_data
WHERE
    -- Optional: Exclude records where transaction count is 0 (like the 'None' row)
    transaction_id IS NOT NULL 
GROUP BY
    Order_YearMonth 
HAVING
    Total_Monthly_Transactions > 0 -- Ensure we don't divide by zero
ORDER BY
    Order_YearMonth DESC;
"""



In [17]:
analysis_df_aov = pd.read_sql_query(sql_query_aov, conn)
print(analysis_df_aov)

   Order_YearMonth  Total_Monthly_Revenue  Total_Monthly_Transactions  \
0          2024-01                 339442                           9   
1          2023-12                5249987                         166   
2          2023-11                5125709                         174   
3          2023-10                5886414                         178   
4          2023-09                5057387                         170   
5          2023-08                4825701                         166   
6          2023-07                5129904                         189   
7          2023-06                4100112                         162   
8          2023-05                3902263                         141   
9          2023-04                4989315                         161   
10         2023-03                5241364                         144   
11         2023-02                4698929                         176   
12         2023-01                4628608          

In [24]:
import pandas as pd

# NOTE: Ensure the 'conn' object is active by re-running connection steps first

sql_query_gender = """
SELECT
    gender AS Customer_Gender,
    
    -- Aggregate Revenue and Transaction Volume
    SUM(prce * quantiy) AS Total_Gender_Revenue,
    COUNT(DISTINCT transaction_id) AS Total_Gender_Transactions,
    
    -- Calculate AOV for the gender group
    CAST(SUM(prce * quantiy) AS REAL) / COUNT(DISTINCT transaction_id) AS Average_Order_Value
    
FROM
    sales_data
GROUP BY
    Customer_Gender
ORDER BY
    Total_Gender_Revenue DESC;
"""


analysis_df_gender = pd.read_sql_query(sql_query_gender, conn)

print("--- Sales Analysis by Gender ---")
print(analysis_df_gender)

--- Sales Analysis by Gender ---
  Customer_Gender  Total_Gender_Revenue  Total_Gender_Transactions  \
0            Male              14938253                        513   
1               F              14821961                        479   
2          Female              14814778                        504   
3               M              14593105                        503   
4            None                 66190                          1   

   Average_Order_Value  
0         29119.401559  
1         30943.551148  
2         29394.400794  
3         29012.137177  
4         66190.000000  


In [25]:
sql_query_category = """
SELECT
    product_category AS Category,
    
    -- Aggregate Revenue and Transaction Volume
    SUM(prce * quantiy) AS Total_Category_Revenue,
    COUNT(DISTINCT transaction_id) AS Total_Category_Transactions,
    
    -- Calculate AOV for the category
    CAST(SUM(prce * quantiy) AS REAL) / COUNT(DISTINCT transaction_id) AS Average_Order_Value
    
FROM
    sales_data
WHERE
    -- Use the same date logic to exclude bad dates if necessary, 
    -- but usually not needed for category analysis unless date is NULL.
    purchase_date IS NOT NULL
GROUP BY
    Category 
HAVING
    Total_Category_Transactions > 0 
ORDER BY
    Total_Category_Revenue DESC; -- Rank categories by total revenue
"""


analysis_df_category = pd.read_sql_query(sql_query_category, conn)
print(analysis_df_category)

      Category  Total_Category_Revenue  Total_Category_Transactions  \
0  Accessories                10365306                          327   
1     Clothing                10199579                          355   
2        Books                 9932469                          332   
3    Furniture                 9659478                          346   
4  Electronics                 9537670                          308   
5    Groceries                 9480633                          332   

   Average_Order_Value  
0         31698.183486  
1         28731.208451  
2         29917.075301  
3         27917.566474  
4         30966.461039  
5         28556.123494  


In [32]:


sql_query = """
SELECT
    -- 1. Combine Year and Month for distinct grouping
    STRFTIME('%Y-%m', purchase_date) AS Order_YearMonth,
    
    -- 2. Aggregate Total Revenue (Price * Quantity)
    SUM(prce * quantiy) AS Total_Monthly_Revenue,
    
    -- 3. Aggregate Transaction Volume (Count of unique transactions)
    COUNT(DISTINCT transaction_id) AS Total_Monthly_Transactions
FROM
    sales_data  -- Assuming your table name is 'sales_data'
GROUP BY
    Order_YearMonth 
ORDER BY
    Order_YearMonth DESC;
"""


try:
    analysis_df = pd.read_sql_query(sql_query, conn)
    
    print("\n--- Monthly Revenue and Transaction Volume Analysis ---")
    print(analysis_df)

except Exception as e:
    print(f"An error occurred during query execution: {e}")
    print("\nPlease ensure your database connection ('conn') is active and the table 'sales_data' is loaded.")
print("\n--- Corrected SQL Analysis Results ---")
print(analysis_df)


--- Monthly Revenue and Transaction Volume Analysis ---
  Order_YearMonth  Total_Monthly_Revenue  Total_Monthly_Transactions
0            None               59234287                        2000

--- Corrected SQL Analysis Results ---
  Order_YearMonth  Total_Monthly_Revenue  Total_Monthly_Transactions
0            None               59234287                        2000


In [30]:


top_sales_query = """
SELECT
    -- 1. Date conversion and extraction (YYYY-MM)
    STRFTIME(
        '%Y-%m', 
        SUBSTR(purchase_date, 7, 4) || '-' || SUBSTR(purchase_date, 4, 2) || '-' || SUBSTR(purchase_date, 1, 2)
    ) AS Order_YearMonth,
    
    -- 2. Aggregate Revenue
    SUM(prce * quantiy) AS Total_Monthly_Revenue
FROM
    sales_data
GROUP BY
    Order_YearMonth 
HAVING
    Total_Monthly_Revenue > 0
ORDER BY
    Total_Monthly_Revenue DESC
LIMIT 3;
"""


try:
    top_three_df = pd.read_sql_query(top_sales_query, conn)
    print("\n--- Top 3 Months by Total Revenue ---")
    print(top_three_df)
    
except NameError:
    print("Error: Name 'conn' is not defined. Please re-run the initial connection and data loading steps.")
except Exception as e:
    print(f"An error occurred: {e}")


--- Top 3 Months by Total Revenue ---
  Order_YearMonth  Total_Monthly_Revenue
0         2023-10                5886414
1         2023-12                5249987
2         2023-03                5241364
